In [4]:
%run init_notebookspace.py
from settings import POST_EST_DIR

DATA_DIR is existant under: C:\Users\LukasGrahl\Documents\GIT\memoire1\data


In [5]:
from src.utils import get_most_recent_mod_output
from src.bayesian import get_xarr_InferenceData

import xarray as xr
import numpy as np

import arviz as az
import pandas as pd

In [6]:
import os

In [11]:
def load_trace_data(file_path: str):
    if 'mod' in file_path:
        return xr.open_dataset(file_path)
    elif 'bvar' in file_path:
        return az.from_netcdf(file_path)
    else:
        raise ValueError('Unknown file tpye')

In [42]:
from config import mod4_params, mod4_priors, mod5_params, mod5_priors, mod6_params, mod6_priors
mods = {
    'mod4_rbc_vanilla': {'is_mod': True},
    'mod5_nk_vanilla': {'is_mod': True},
    'mod6_nk_energy_lin2': {'is_mod': True},
    'bvar': {'is_mod': False},
}

# load model
for key in mods.keys():
    file_name = get_most_recent_mod_output(POST_EST_DIR, f'{key}_accepted')
    mods[key]['post_out'] = load_trace_data(os.path.join(POST_EST_DIR, file_name))

In [43]:
def get_xarr_InferenceData(xarr_in: xr.Dataset):
    # xarr = xarr_in.where(xarr_in.is_solved).copy()
    draw = len(xarr.draw)

    xr_prior = xr.Dataset(
        {
            key: (
                ['chain', 'draw', 'a_dim'],
                np.concatenate(
                    [
                        np.array([0] * draw).reshape(draw, 1),
                        np.array(xarr.sel(parameter=key).prior).reshape(draw, 1),
                        np.array([0] * draw).reshape(draw, 1)
                    ],
                    axis=1
                ).reshape(1, draw, 3)
            )
            for key in xarr.parameter.values
        },
        coords={
            "chain": (["chain"], np.arange(1)),
            "draw": (["draw"], np.arange(draw)),
            "a_dim": (["a_dim"], ["x", "y", "z"])

        }
    )

    xr_post = xr.Dataset(
        {
            key: (
                ['chain', 'draw', 'a_dim'],
                np.concatenate(
                    [
                        np.array([0] * draw, dtype=float).reshape(draw, 1),
                        np.array(xarr.sel(parameter=key).posterior, dtype=float).reshape(draw, 1),
                        np.array([0] * draw, dtype=float).reshape(draw, 1)
                    ],
                    axis=1
                ).reshape(1, draw, 3)
            )
            for key in xarr.parameter.values
        },
        coords={
            "chain": (["chain"], np.arange(1)),
            "draw": (["draw"], np.arange(draw)),
            "a_dim": (["a_dim"], ["x", "y", "z"])

        }
    )

    xr_loglike = xr.Dataset(
        {
            'obs': (
                ['chain', 'draw', 'a_dim'],
                np.concatenate(
                    [
                        np.array([0.] * draw, dtype=float).reshape(draw, 1),
                        np.array(xarr.log_likelihood, dtype=float).reshape(draw, 1),
                        np.array([0.] * draw, dtype=float).reshape(draw, 1)
                    ],
                    axis=1
                ).reshape(1, draw, 3)
            )
        },
        coords={
            "chain": (["chain"], np.arange(1)),
            "draw": (["draw"], np.arange(draw)),
            "a_dim": (["a_dim"], ["x", "y", "z"])

        }
    )
    return xr_prior, xr_post, xr_loglike

In [45]:
for key in mods.keys():
    xarr = mods[key]['post_out']  
    if key == 'mod4_rbc_vanilla':
        continue
    xarr
    # if mods[key]['is_mod']:
    #     mods[key]['az_inf'] = az.InferenceData(**dict(zip(
    #         ['prior', 'posterior', 'log_likelihood'], 
    #         *[get_xarr_InferenceData(xarr)]
    #     )))
    # else:
    #     mods[key]['az_inf'] = mods[key]['post_out']

In [46]:
xarr

Inference data with groups:
	> posterior
	> sample_stats
	> observed_data

In [34]:
mods['mod4_rbc_vanilla']['az_inf']

Inference data with groups:
	> posterior
	> log_likelihood
	> prior

In [33]:
az.compare(
    {
        'mod4': mods['mod4_rbc_vanilla']['az_inf'], 
        'mod5': mods['mod5_nk_vanilla']['az_inf'], 
        'mod6': mods['mod6_nk_energy_lin2']['az_inf'],
        # 'bvar': mods['bvar']['az_inf']
    },
    ic='loo')

C:\Users\LukasGrahl\miniconda3\envs\env_conda3_memoire1\lib\site-packages\arviz\stats\stats.py:803: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
C:\Users\LukasGrahl\miniconda3\envs\env_conda3_memoire1\lib\site-packages\arviz\stats\stats.py:803: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
C:\Users\LukasGrahl\

,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
mod5,0,-2.034194e+05,1.174170e+03,0.000000e+00,0.333333,1.660913e+05,0.000000e+00,True,log
mod6,1,-2.228546e+06,1.636106e+06,2.025127e+06,0.333333,1.819600e+06,1.653509e+06,True,log
mod4,2,-2.263503e+06,1.863774e+06,2.060084e+06,0.333333,1.848142e+06,1.682051e+06,True,log
